# Entry G12: Degree Comparison

This is essentially a redo of the unweighted degree metrics from [Entry G10](https://julielinx.github.io/blog/g10_local_metrics/) where I ran the same metrics and queries using the multigraph as created in [Entry G11](https://julielinx.github.io/blog/g11_create_multigraphdb_desktop/). The change to the multigraph allowed me to compare the metrics across graph models. This will be a crutial aspect once I start comparing runtime of the different queries.

For now, it also allows for much easier comparison between the results of the different graph models. For example, I can now see that there are a different number of `KNOWS` relationships in the unimodal model and the mixed model, which shouldn't be the case; these models should have the exact same number of `KNOWS` relationships.

I made a few other changes, one of which was adding an additional way to grab the relationship count using the `size` function. Another change was to start puting the data into dataframes/tables to more easily compare the final results.

One of the best examples of the usefulness of this was in double checking the bimodal model's pattern match of outgoing and incoming relationships. I played with leaving the label off vs including it.

`MATCH (c1)
OPTIONAL MATCH (c1)-[]->(c2)`
vs
`MATCH (c1:Hero)
OPTIONAL MATCH (c1)-[]->(c2)`

While the minimum and maximum results were the same, the average and standard deviation were quite different. If we think through the cause, it becomes apparent why this is true. In using a directed relationship we can only traverse from `Hero` nodes to `Comic` nodes and not the reverse. This means that all `Comic` nodes return a degree of zero.

<img src='images/directed_rel.png'>

#### `gds` function

Relationship orientation options:

- `NATURAL`
  - Follows the indicated relationship direction
  - Ex: (:Hero)-[:APPEARS_IN]->(:Comic)
  - For the bimodal model, this will tell us the number of comics that a hero appears in
- `REVERSE` 
  - Reverses the indicated relationship direction
  - Ex: (:Hero)<-[:APPEARS_IN]-(:Comic)
  - For the bimodal model, this will tell us the number of heros in any given comic
- `UNDIRECTED`
  - Uses relationship regardless of direction
  - Ex: (:Hero)-[:APPEARS_IN]-(:Comic)
  - This will include both the number of comics that a hero appears in and the number of heros in any given comic
  
*Caution*, upon comparison with the other methods, it appears that the gds degree function has some important limitations. Notably, I was unable to determine how to indicate we only want to count the degree of a specific label where we include nodes not of the same label. Example, I want to know how many comics a hero is in, but I don't want to count the comic nodes. With the `Reverse` parameter, we exclude the Comic to Hero relationships, but still count the Comic nodes themselves as 0 degree.

The `nodeProjection` parameter allows limiting the labels, but it won't count any label not listed. Once listed, all of those nodes are included in the degree count. For example, if I just want 